# 5.1 전자상거래 쿼리

## 5.1.1 상품, 카테고리, 리뷰

In [1]:
import pymongo
connection = pymongo.MongoClient('localhost', 27017)
db = connection.test

In [2]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [9]:
db.products.insert({'slug':'wheelbarrow-9092','sku':'9092','name':'Extra Large Wheelbarrow',
                    'description':'Heavy duty wheelbarrow...', 'details':{'weight' : 47, 
                                                                         'weight_units':'lbs',
                                                                        'model_num':4039283402,
                                                                         'manufacturer':'Acme',
                                                                         'color':'Green'},
                   'total_review':4,
                   'average_review':4.5,
                   'pricing':{'retai':589700,'sale':489700},
                   'price_history': [{
                       'retail':529700,
                       'sale':429700,
                       'start':'new Date(2010,4,1)',
                       'end':'new Date(2010, 4,8)',
                   },
                       {
                       'retail':529700,
                       'sale':529700,
                       'start':'new Date(2010,4,9)',
                       'end':'new Date(2010, 4,16)',
                   },
                       
                   ]})

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ObjectId('5ed10e56aa71508dada5482c')

In [15]:
# 상품 페이지가 행당 상품의 슬러그로 액세스 된다고 가정할 때, 밑의 쿼리로 필요한 모든 데이터를 얻을 수 있다.
print(db.products.find_one({'slug':'wheelbarrow-9092'}))

{'_id': ObjectId('5ed10e56aa71508dada5482c'), 'slug': 'wheelbarrow-9092', 'sku': '9092', 'name': 'Extra Large Wheelbarrow', 'description': 'Heavy duty wheelbarrow...', 'details': {'weight': 47, 'weight_units': 'lbs', 'model_num': 4039283402, 'manufacturer': 'Acme', 'color': 'Green'}, 'total_review': 4, 'average_review': 4.5, 'pricing': {'retai': 589700, 'sale': 489700}, 'price_history': [{'retail': 529700, 'sale': 429700, 'start': 'new Date(2010,4,1)', 'end': 'new Date(2010, 4,8)'}, {'retail': 529700, 'sale': 529700, 'start': 'new Date(2010,4,9)', 'end': 'new Date(2010, 4,16)'}]}


find와 find_one의 차이점
<br>
findone은 도큐먼트를 반환하고, find는 커서 객체를 반환한다. 위의 find_one은 db.products.find({'slug':'wheelbarrow-9092'}).limit(1)과 같다 
<br>
하나의 도큐먼트를 얻고자 하면 findOne 매서드는 도큐먼트가 존재할 경우 그 도큐먼트를 반환한다. 여러 도큐먼트를 반환해야될 경우 find 매서드를 사용해서 애플리케이션에서 커서를 반복해야한다.
<br>
만약 findOne에 매칭되는 항목이 여러개면 정렬상 가장 첫번째 항목을 반환하게 된다. 

## skip, limit 그리고 쿼리 옵션
대부분의 애플리케이션에서는 리뷰를 페이지로 나누게 된다. 이를 위해 MongoDB는 skip과 limit 옵션을 사용한다


In [ ]:
db.reviews.find({'product_id':product['_id']}).skip(0).limit(12)

skip과 limit는 질의 뒤에 호출된 것처럼 보이지만, 정렬과 제한 매개변수는 질의와 함꼐 전달되고 몽고디비 서버에서 처리하게 된다. 이를 매서드 체이닝이라 부르며 쿼리 작성을 쉽게 하기 위한 것이다. 

In [ ]:
# 몽고디비에게 추천수가 많은 순서대로 정렬해서 처음 12개의 리뷰를 반환할 것을 요청한 것이다. 
db.reviews.find({'product_id':product['id']}).sort({'helpful_votes':-1}).limit(12)

In [ ]:
# 상품의 쿼리 페이지
page_number = 1
product = db.products.findOne({'slug':'wheel-barrow-9092'})
category = db.categories.findOne({'_id':product['main_cat_id']})
reviews_count = db.reviews.count({'product_id':product['_id']})
reviews = db.reviews.find({'product_id':product['_id']}).skip((page_number - 1) * 12).limit(12).sort({'helpful_votes':-1})


## 5.1.2 사용자와 주문

In [ ]:
# id 로그인시 사용하는 쿼리
# 밑의 결과가 맞다면 사용자 도큐먼트 결과값을 받는다. 그렇지 않으면 아무것도 반환하지 않는다. 
db.users.findOne({'username':'user','hashed_password':'fsadfgsda'})

In [ ]:
# 이렇게 사용하면 반환 필드를 제한할 수 있다. 
# RDBMS에 익숙하다면 select * 과 select id의 차이로 생각하면 됨.
db.users.findOne({'username':'user','hashed_password':'fsadfgsda'}, {'_id': 1})

In [ ]:
# 정규식으로도 찾을 수 있다. 
db.users.find({'last_name':'/^Ba/'})

In [ ]:
# 다음과 같이 특정 범위를 질의할 수 있다.
# $gt는 보다 큰, $lt는 보다 작은
db.users.find({'addresses.zip':{'$gt':10019,'$lt':10040}})

# 5.2 MongoDB의 쿼리 언어

## 5.2.1 질의 조건과 셀렉터
가장 간단한 쿼리는 키-값 쌍이 도큐먼트에 문자적으로 일치하는 셀렉터를 사용하는 쿼리다. 


In [ ]:
db.users.find({'last_name':'Hanker'})
db.users.find({'first_name':'Smith', 'birth_year':1975})

In [16]:
# 몽고디비는 대소문자를 구분한다. 구분하지 않기 위해서는 정규식을 사용해야된다. 

연산자 요약
<br>
$lt = 보다 작은

$gt = 보다 큰

<br>
$lte = 보다 작거나 같은


$gte = 보다 크거나 같은
<br>
문자열에도 이 쿼리를 사용할 수 있다.

집합 연산자 요약

$in = 어떤 인수든 하나라도 참고 집합에 있는 경우 일치

$all = 모든 인수가 참고 집합에 있고 배열이 포함된 도큐먼트에서 사용되는 경우 일치

$nin = 그 어떤 인수도 참고 집합에 있지 않을 경우 일치


In [ ]:
# 사용 예
db.products.find({'main_cat_id': {'$in':['hbadfsd','dfsdaf','dfasdf']}})

선택성은 인덱스를 사용하여 결과의 범위를 좁히는 쿼리의 기능이다. $in과 $all은 인덱스를 쓰지만, $nin은 인덱스를 사용하지 않기 때문에 컬렉션을 스캔한다. 

논리 연산자 요약

$ne = 인수가 요소와 같지 않은 경우 일치

$not = 일치 결과를 반전시킴(반대로 만듦)

$or = 제공된 검색어 집합 중 하나라도 True인 경우 일치

$nor = 제공된 검색어 집합 중 그 어떤 것도 True가 아닌 경우 일치

$and = 제공된 검색어 집합이 모두 True인 경우 일치

$exists = 요소가 도큐먼트 안에 존재할 경우 일치

In [ ]:
# 사용 예
db.products.find({'details.manufacturer':'Acme','tags':{$ne:'gardening'}})

In [ ]:
# exists 사용 예
db.products.find({'details.color':{$exists:false}})
db.products.find({'details.color':{$exists:true}})

In [ ]:
# 서브 도큐먼트 매칭
db.products.find({'details.manufacturer':'Acme'})
db.products.find({'details.manufacturer.id':432})

In [17]:
# 키의 순서가 매우 중요하다. 따라서 쿼리를 만들 때 순서를 고려해서 만들도록 하자. 

배열 연산자 요약

$elemMatch = 제공된 모든 조건이 동일한 하위 도큐먼트에 있는 경우 일치

$size = 배열 하위 도큐먼트의 크기가 제공된 리터럴 값과 같으면 일치

In [ ]:
# 여러 개의 조건을 하나의 서브 도큐먼트에 대해 제한하려면 $elemMatch 연산자를 쓰면 된다. 
db.users.find({'addresses':'$elemMatch':{
    'name':'home',
    'state':'NY'
}})

In [ ]:
# 배열은 크기별로 질의 가능하다.
db.users.find({'addresses':{$size:3}})

In [ ]:
# 자바스크립트로도 쿼리를 표현할 수 있다. 
# 자바스크립트 표현식을 넘겨주기 위해서는 $where 연산자를 사용한다. 
db.reviews.find({
    '$where':'function() {return this.helpful_votes > 3;}'
})
# 간단버전
db.reviews.find({'$where':'this.helpful_votes>3'})

자바스크립트는 오베헤드가 많이 발생한다.(인덱스도 사용불가) 따라서 표준 쿼리 언어로 쿼리를 표현할 수 없는 경우에만 사용한다. 자바스크립트 사용시 인젝션 공격에 노출되는 위험성에 대해 알고 있어야 한다. 인젝션 공격은 사용자가 자바스크립트 쿼리에 직접 코드를 입력하는 것이 허용되면 언제든지 가능하다. 

In [ ]:
# 정규 표현식 연산자는 $regex 
# 정규표현식을 지원하지 않는 환경에서는 $regex와 $options라는 특수한 연산자를 사용할 수 있다. 
db.reviews.find({'user_id':'ObjectId','text':{'$regex':'best|worst',''}})